In [18]:
import cassiopeia as cass
from cassiopeia import Summoner, Match
from cassiopeia.data import Season, Queue
from collections import Counter
import arrow
import csv


In [19]:

cass.set_riot_api_key("Key") #Get it from Riot Dev Portal https://developer.riotgames.com/

In [23]:
unpulled_summoners = []
pulled_matches = []
match_player_dict = {} 
matchcounter = 0

In [33]:
"""#read in unpulled summoners
with open('unpulled_summoners.csv', "r") as f:
    reader = csv.reader(f)
    test_unpulled_summoners = list(reader)

testlist = []
for elem in test_unpulled_summoners:
    for elem2 in elem:
        testlist.append(elem)

print(testlist[5])
print(unpulled_summoners[5])

with open('match_playerPUUIDs_timestamp.csv', "r") as f:
    reader = csv.reader(f)
    test_pulled_matches = [line[0] for line in reader]"""

['UQ_YVz0C7Nk8YLmQ5GytpGUwvLvSXKJwiQ5CjrZjfF0cRmBnmq2iOGbXkGU1uTz6MsLrOE0-V3D4yg']
UQ_YVz0C7Nk8YLmQ5GytpGUwvLvSXKJwiQ5CjrZjfF0cRmBnmq2iOGbXkGU1uTz6MsLrOE0-V3D4yg


11074


In [36]:
#start_summoner = Summoner(name="Lahmacun Döner", region="EUW")
#unpulled_summoners.append(start_summoner.puuid)

# A MatchHistory is a lazy list, meaning it's elements only get loaded as-needed.
while unpulled_summoners:

    # Setting new Summoner to mine the Match History of
    new_puuid = unpulled_summoners[0]
    new_summoner = Summoner(puuid=new_puuid, region="EUW")

    try:
        tier = str(new_summoner.league_entries.fives.tier)
    
    #Bug with RIOT API If the player is ranked in TFT Pair mode, it will send a false response and crash cassiopeia MAAN WHY HAS EVERYTHING BE SO BUGGY AND NOT UP TO DATE BUG IS KNOWN IN GITHUB SINCE THE MODE CAME OUT THANKS RIOT
    except ValueError:
        print("Value Error in Tier, known bug of the API, saving tier of this Summoner as Unknown")
        tier = "Unknown"


    match_history = cass.get_match_history(
        continent=new_summoner.region.continent,
        puuid=new_summoner.puuid,
        begin_index = 0,
        end_index = 100)

    #Iterating through the Match History of the Summoner
    for match in match_history:

        #Checking if we maybe already mined the specific match in another summoners Match History
        if match.id not in pulled_matches:
            pulled_matches.append(match.id)
            participant_ids = []
            matchcounter += 1
            print(matchcounter, match.creation)        

            
            #Iterating trough the partcipiants in the Match
            for p in match.participants:
                  #There are some caching bugs with cassiopeia where sometimes in the participants are saved twice, so a match has 20 instead of 10 and then the same 10 again. So we check for that and then just cut the doubles at the end 
                  if len(participant_ids) >= 10:
                    break
                        
                  #if (not p.is_bot) and p.summoner.exists: Better Check but is expensive since it takes extra Calls, so we do the check under here
                  #Not rly the best check, but does the job and requieres no extra calls to the API
                  if p.summoner is not None and p.summoner.puuid is not None:
                    
                    if p.summoner.puuid not in unpulled_summoners:
                        unpulled_summoners.append(p.summoner.puuid)


                    participant_ids.append(p.summoner.puuid)
                #entries, rank_last_season, ranks
            timestamp = str(match.creation)
            match_player_dict[match.id] = (participant_ids, timestamp)
        
    unpulled_summoners.remove(new_summoner.puuid)


    #I know it takes a bit of performance to the saving for every Summoner we pulled the 20 matches for, but its negligable in comparison to the API Calls and waiting time for the rate limits. 

    #Save the Dictionary of matches including the player ids and a Timestamp
    with open("match_playerPUUIDs_timestamp.csv","a") as file:
        for key, value in match_player_dict.items():
            line = f"{key}, {value}\n"
            file.write(line)

    match_player_dict.clear() 

    #Saves the fully mined Summoners and their ranks
    with open("fullyMinedSumonerPUUID_Rank.csv","a") as file:
            line = f"{new_puuid},{tier}\n"
            file.write(line)


    #dumps the unpulled summoner list, so we can pick up again where we left off or it crashed or smth
    with open("unpulled_summoners.csv","w") as file:
        wr = csv.writer(file, delimiter="\n")
        wr.writerow(unpulled_summoners)



Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/I3WzD9YyC7-m4NjFVVvjm0zd_nwzYBlVnZ6S1xS_-i2K-WkVdWJmxeRYZq-NheWmEmsJSspWDQHKrQ/ids?beginIndex=0&endIndex=100
25832 2022-05-07T21:41:00+00:00
25833 2022-05-07T21:17:39+00:00
25834 2022-05-07T16:13:40+00:00
25835 2022-05-07T15:46:50+00:00
25836 2022-05-07T10:22:10+00:00
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_5858658353
25837 2022-05-06T16:50:20+00:00
Making call: https://euw1.api.riotgames.com/lol/league/v4/entries/by-summoner/kV1Asy1v1i40VU1fUJ-AEbs54a9E33tUeLGe_dg6ImVsgIA
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/MkGLy6sIc85WdO-HgKryF7hFbEC6ayKCIKvD41D-6GaxnDxY_-MJouxKnoP955UuoCymq7LZcuyPVQ/ids?beginIndex=0&endIndex=100
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_5860875019
25838 2022-05-07T21:05:14+00:00
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_5860520031
25839 2022-05-07T18:09:46+00:00

KeyError: 400

In [5]:
print(len(match_player_dict))

51


In [9]:
for key in match_player_dict:
    print(key, match_player_dict[key])

5846005280 (['b3m8FtWaOV9RTL6EL16KnwyfN8Uvxgrzu0kt1RFZRA0OuFhfzI2WL3XZ2SVC26WG43DWp1sORKLPEg', 'IiC6jy4fxvf3qiA6vPIFjJ57KLgIzydeX1hXy8bseYDwqq5Vr5ZpTQUwqc_zD96ju-fAgaJFIEkmsg', 'bsRmJj-JhGKjPgJIKIDhtKmRmHtw8AEDbtfs4ypwLr65bvzZOiY2PZ4QGAEtnWjeQRsYpBvZ-gPuFA', 'EsiYGe2O7c2WQuyinxKixQXaQ52xdSyMVzdMQ6Uj5cPsgaOVoDPHBsPaFidp0bAsW3GUI1MKjVxGTA', 'GwyWTqkR5hfu1ZwOAL2LXyXwOxd2vFS-5dxoYk1-5qrj-ONF8E1nynscOaVkHb5FEA-4yGOjWDc3AA', 'AsOHEhnYzpvrADPrKvpzhvf_x3GXtNVL5FrY-KFcWGHxX43zI1LjwxkVDKODPQLu1gWF0Z6wP68llg', 'LhLcSIe-tXg6E9I66e0TSI5-yCMo4hIvhC1yQlIZnB86T6I-R1Ern7xrEuKSjiWxULnGfW-TS-uSmg', 'DUi-xPkeFKUDiT0EqfJoPlhs1gXXavbsVIhJO15XqQLUrqvOKChSys7qE0ighnFzjJtFPD-hqbaPmQ', 'SEwkgutzWJG8UZOb0TY4xcRqw1pwa7L7jKMX1GIam1EMKT7w-Fq-ne_ivsAFFGXvRLEXkugqHEn0hA', 'ykODW8V-LFKuYSTMx2HgGKkOc6Mettha9KPChSbPgvkCBY_eS5vOn_2Ppkn3ZRJpaSqdyJGhjY_spg'], '2022-04-28T18:37:23+00:00')
5845927224 (['KxhKe3A7R-wyHlnczR5hY_HaGcVOsEq2KpAQkcYFtUOOcrGQclN9Abbnh3xLpQt7rb8ejQHMNSnNGg', 'LhLcSIe-tXg6E9I66e0TSI5-yCMo4hIvhC1yQlIZn